In [1]:
import os, sys
import yaml
import json
import numpy as np
import secrets  # for generating random hex code
import importlib.util  # for importing a model

import tensorflow as tf
from data_processing.Processor import load_detection_ds
from models.fcos.interpreter import Interpreter

CONFIG_PATH = "training/detector_config.yaml"

In [2]:
def numpy_json_converter(obj):
    """
    Allows to dump numpy arrays as json.
    """
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, datetime.datetime):
        return obj.__str__()

In [3]:
with open(CONFIG_PATH, "r") as fd:
    config = yaml.safe_load(fd)

config

{'model': {'name': 'fcos',
  'build': 'build',
  'args': {'input_h': 592,
   'input_w': 800,
   'l2_reg': 1e-05,
   'dropout': 0,
   'strides': [8],
   'regression_weight': 0.05}},
 'heads_names': ['concatenate', 'concatenate1'],
 'ext_keys': ['img', 'target0'],
 'data': {'train': 'training/train_dataset.yaml',
  'val': 'training/val_dataset.yaml'},
 'optimizer': {'name': 'Adam', 'lr': 0.001},
 'callbacks': {'lr_scheduler': {'factor': 0.5, 'patience': 10, 'cooldown': 5},
  'early_stop': {'patience': 25}},
 'epochs': 200,
 'batch_size': 8}

In [4]:
# Keyword arguments for building a model
kwargs = config["model"]["args"]

# importing
module_path = os.path.join(
    "models", config["model"]["name"], config["model"]["build"] + ".py"
)
spec = importlib.util.spec_from_file_location(config["model"]["build"], module_path)
build = importlib.util.module_from_spec(spec)
spec.loader.exec_module(build)

In [5]:
# preprocessing
train_data_path = config["data"]["train"]
val_data_path = config["data"]["val"]

train_raw_ds = load_detection_ds(train_data_path)
val_raw_ds = load_detection_ds(val_data_path)

processor = build.build_processor(**kwargs)
train_processor = build.build_train_processor(**kwargs)

with open(train_data_path,'r') as train_fp:
    training_list = yaml.safe_load(train_fp)
train_len = len(training_list['white_list'])

train_processed_ds = (
    train_raw_ds.shuffle(train_len,
                         reshuffle_each_iteration=True
                        ).map(train_processor).batch(config["batch_size"]).prefetch(4)
)
val_processed_ds = val_raw_ds.map(processor).batch(config["batch_size"]).prefetch(4)

In [6]:
# building a model
model = build.build_model(**kwargs)
loss = build.build_loss(**kwargs)
metrics = build.build_metrics(**kwargs)

# compiling
optimizer = getattr(tf.keras.optimizers, config["optimizer"]["name"])
model.compile(
    optimizer=optimizer(config["optimizer"]["lr"]), loss=loss, metrics=metrics,
)

In [7]:
# generating model code
model_code = secrets.token_hex(16)

# callbacks
model_save_path = os.path.join(
    "models", config["model"]["name"], "snapshots", model_code
)
os.makedirs(model_save_path, exist_ok=True)
print(model_save_path)

models/fcos/snapshots/ac47c98a3557eb58a7092b49055eaf33


In [8]:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=config["callbacks"]["lr_scheduler"]["factor"],
    patience=config["callbacks"]["lr_scheduler"]["patience"],
    cooldown=config["callbacks"]["lr_scheduler"]["cooldown"],
    min_lr=1e-7,
    verbose=True,
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=config["callbacks"]["early_stop"]["patience"]
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(model_save_path, "weights.hdf5"),
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    mode="min",
)

callbacks = [lr_scheduler, early_stop, checkpoint]

In [9]:
tf.__version__

'2.3.1'

In [ ]:
# training
history = model.fit(
    train_processed_ds,
    epochs=5, #config["epochs"],
    validation_data=val_processed_ds,
    callbacks=callbacks
)

# saving history
with open(os.path.join(model_save_path, "history.json"), "w") as fd:
    history.history["train_config"] = config
    json.dump(history.history, fd, indent=4, default=numpy_json_converter)

Epoch 1/5
